In [ ]:
# The following is to know when this notebook has been run and with which python version.
import time, sys
print(time.ctime())
print(sys.version.split('|')[0])

# E Introduction to Scipy

This is part of the Python lecture given by Christophe Morisset at IA-UNAM.

Scipy is a library with a lot of foncionalities, we will not cover everything here, but rather point to some of them with examples.
Some useful links about scipy:

* https://scipy-lectures.github.io/intro/scipy.html
* http://docs.scipy.org/doc/scipy/reference/tutorial/

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import scipy # This imports a lot of scipy stuff, but not the "important" modules

### Some usefull methods

In [ ]:
from scipy.special import gamma
%timeit g1 = gamma(10.3)
%timeit g1 = gamma(10)
%timeit g2 = 9*8*7*6*5*4*3*2
%timeit g3 = 10*9*8*7*6*5*4*3*2
g1 = gamma(10.3)
g2 = 9*8*7*6*5*4*3*2
g3 = 10*9*8*7*6*5*4*3*2
print(g1, g2, g3)

In [ ]:
from scipy import constants as cst
print(cst.astronomical_unit) # A lot of constants
from scipy.constants import codata # a lot more, with units. From NIST
print('{} {}'.format(codata.value('proton mass'), codata.unit('proton mass')))

List there: http://docs.scipy.org/doc/scipy/reference/constants.html#constants-database

### Integrations

In [ ]:
from scipy.integrate import trapz, cumtrapz, simps
#help(scipy.integrate) # a big one...
print('----------------------------------------------------------------------------------')
help(trapz)
print('----------------------------------------------------------------------------------')
help(cumtrapz)
print('----------------------------------------------------------------------------------')
help(simps)

In [ ]:
dir(scipy.integrate)

In [ ]:
# Defining x and y
x = np.linspace(0, np.pi, 100)
y = np.sin(x)
# Compare the integrales using two methods
%timeit i1 = trapz(y, x)
%timeit i2 = simps(y, x)

print(trapz(y, x))
print(simps(y, x))

x = np.linspace(0, np.pi, 10)
y = np.sin(x)
%timeit i1 = trapz(y, x)
%timeit i2 = simps(y, x)
print(trapz(y, x))
print(simps(y, x))


In [ ]:
# Cumulative integrale
cum = cumtrapz(np.abs(y), x)
print(len(x), len(cum), cum)

In [ ]:
# Cumulative integral
print('{} {}'.format(len(x), len(cumtrapz(np.abs(y), x))))
f, ax = plt.subplots()
ax.plot(x[0:-1], cumtrapz(np.abs(y), x), 'bo');

In [ ]:
from scipy.integrate import quad # To compute a definite integral
from scipy.special import jv # Bessel function
%timeit res = quad(np.sin, 0, np.pi)
print(quad(np.sin, 0, np.pi))
#help(quad)
print(quad(lambda x: jv(3.5, x), 0, 10)) # Integrate the Bessel function of order 2.5 between 0 and 10

We now want to evaluate:
$$ \int_0^1 1 + 2 x + 3 x^2 dx $$

In [ ]:
# We want here integrate a user-defined function (here polynome) between 0 and 1
def f(x, a, b, c):
    """ Returning a 2nd order polynome """
    return a + b * x + c * x**2
%timeit I = quad(f, 0, 1, args=(1,2,3)) # args will send 1, 2, 3 to f
I = quad(f, 0, 1, args=(1,2,3)) # args will send 1, 2, 3 to f
print(I)
Integ = I[0]
print(Integ)

### Interpolations

In [ ]:
from scipy.interpolate import interp1d, interp2d, splrep, splev, griddata

In [ ]:
#help(scipy.interpolate) # a huge one...
help(interp1d)

In [ ]:
x = np.linspace(0, 10, 10)
y = np.sin(x)
f = interp1d(x, y) # this creates a function that can be call at any interpolate point
f2 = interp1d(x, y, kind='cubic') # The same but using cubic interpolation
tck = splrep(x, y, s=0) # This initiate the spline interpolating function, finding the B-spline representation of 1-D curve.
# tck is a sequence of length 3 returned by `splrep` or `splprep` containing the knots, coefficients, and degree of the spline.
f3 = lambda x: splev(x, tck) # Evaluate the B-spline or its derivatives.

In [ ]:
# Defining the high resolution mesh
xfine = np.linspace(0, 10, 100)
yfine = np.sin(xfine)
# Plot to compare the results
fig, (ax1, ax2) = plt.subplots(2, figsize=(10,10))

ax1.plot(x, y, 'or', label='data')
ax1.plot(xfine, f(xfine), label='lineal')
ax1.plot(xfine, f2(xfine), label='cubic')
ax1.plot(xfine, f3(xfine), label='spline', alpha=0.6) 
ax1.legend(loc=9)

ax2.plot(xfine, (yfine-f(xfine)), label='data-linear')
ax2.plot(xfine, (yfine-f2(xfine)), label='data-cubic')
ax2.plot(xfine, (yfine-f3(xfine)), label='data-spline')
ax2.legend(loc='best')
ax2.set_ylim((-0.03, 0.02));

In [ ]:
x0 = 3.5
print('{} {} {} {}'.format(np.sin(x0), f(x0), f2(x0), f3(x0)))

#### 2D interpolation

In [ ]:
# Defining a 2D-function
def func(x, y):
    return x * (1+x) * np.cos(4*np.pi*x) * np.sin(4*np.pi*y**2)**2

In [ ]:
# Initializing a 2D coordinate grid. Note the use of j to specify that the end point is included.
grid_x, grid_y = np.mgrid[0:1:100j, 0:1:200j]

In [ ]:
print(grid_x)
print(grid_y)

In [ ]:
plt.imshow(grid_y)

In [ ]:
# Generating 1000 x 2 points randomly
points = np.random.rand(1000, 2)
print(points)
values = func(points[:,0], points[:,1])
print(np.min(points), np.max(points))

In [ ]:
# griddata is the 2D-interpolating method. We want to obtain values on (grid_x, grid_y) points, 
# using "points" and "values".
%timeit grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
%timeit grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')
%timeit grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic')

In [ ]:
# 4 subplots
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')
grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic')
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12))

ax1.imshow(func(grid_x, grid_y), extent=(0,1,0,1), interpolation='none',
           origin='upper')
ax1.plot(points[:,0], points[:,1], 'ko', ms=1)
ax1.set_title('Original')

ax2.imshow(grid_z0, extent=(0,1,0,1), interpolation='none',
           origin='upper')
ax2.plot(points[:,0], points[:,1], 'k.', ms=1)
ax2.set_title('Nearest')

ax3.imshow(grid_z1, extent=(0,1,0,1), interpolation='none',
           origin='upper')
ax3.plot(points[:,0], points[:,1], 'k.', ms=1)
ax3.set_title('Linear')

ax4.imshow(grid_z2, extent=(0,1,0,1), interpolation='none',
           origin='upper')
ax4.plot(points[:,0], points[:,1], 'k.', ms=1)
ax4.set_title('Cubic');

In [ ]:
print(grid_z1[10,10])

### Linear algebra

Scipy is able to deal with matrices, solving linear equations, solving linear least-squares problems and pseudo-inverses, finding eigenvalues and eigenvectors, and more, see here: 
http://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html

### Data fit

In [ ]:
from scipy.optimize import curve_fit # this is used to adjust a set of data

In [ ]:
#help(curve_fit)

In [ ]:
def gauss(x, A, B, C, S):
    # This is a gaussian function.
    return A + B*np.exp(-1 * (x - C)**2 / (2 * S**2))

In [ ]:
# We define the parameters used to generate the signal (gaussian at lambda=5007)
N_lam = 200
A = 4.
B = 15.
Lam0 = 5007.
Sigma = 10.
# We define a wavelength range
lam = np.linspace(4900, 5100, N_lam)
# Computing the signal
fl = gauss(lam, A, B, Lam0, Sigma)
f, ax =plt.subplots()
ax.plot(lam, fl)
ax.set_ylim(0,20);

In [ ]:
SN = 2. # Signal/Noise
noise = B / SN * (np.random.rand(N_lam)*2 - 1)
fl2 = fl + noise
f, ax =plt.subplots()
ax.plot(lam, fl, label='signal')
ax.plot(lam, noise, label='noise')
ax.plot(lam, fl2, label='signal + noise')
ax.legend(loc='best');

In [ ]:
# Initial guess:
A_i = 0.
B_i = 1.
Lam0_i = 5000.
Sigma_i = 1.
fl_init = gauss(lam, A_i, B_i, Lam0_i, Sigma_i)
error = np.ones_like(lam) * np.mean(np.abs(noise)) # We define the error (the same on each pixel of the spectrum)

In [ ]:
# fitting the noisy data with the gaussian function, using the initial guess and the errors
fit, covar = curve_fit(gauss, lam, fl2, [A_i, B_i, Lam0_i, Sigma_i], error)
print('{0:.2f} {1:5.2f} {2:.2f} {3:5.2f} {4:5.2f}'.format(A_i, B_i, Lam0_i, Sigma_i, B_i*Sigma_i))
print('{0:.2f} {1:5.2f} {2:.2f} {3:5.2f} {4:5.2f}'.format(A, B, Lam0, Sigma, B*Sigma))
print('{0[0]:.2f} {0[1]:5.2f} {0[2]:5.2f} {0[3]:.2f}  {1:5.2f}'.format(fit, fit[1]*fit[3]))

In [ ]:
# Computing the fit on the lambdas
fl_fit = gauss(lam, fit[0], fit[1], fit[2], fit[3])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

ax1.plot(lam, fl, label='original')
ax1.plot(lam, fl2, label='original + noise')
ax1.plot(lam, fl_init, label='initial guess')
ax1.plot(lam, fl_fit, label='fit')
ax1.legend()

ax2.plot(lam, fl_fit - fl2, label='Residu=Fit-input')
ax2.legend();

In [ ]:
# Integrating using the Simpson method the gaussian (without the continuum)
print(simps(fl - A, lam))
print(simps(fl2 - fit[0], lam))
print(simps(fl_fit - fit[0], lam))

In [ ]:
khi_sq = (((fl2-fl_fit) / error)**2).sum() # The problem here is to determine the error...
khi_sq_red = khi_sq / (len(lam) - 4 - 1) # reduced khi_sq = khi_sq / (N - free_params - 1)
print('khi^2={}, khi^2_reduced={}'.format(khi_sq, khi_sq_red))

### Multivariate estimation

In [ ]:
from scipy import stats

In [ ]:
def measure(n):
    """Measurement model, return two coupled measurements."""
    m1 = np.random.normal(size=n)
    m2 = np.random.normal(scale=0.5, size=n)
    return m1+m2, m1-m2

In [ ]:
m1, m2 = measure(2000)
xmin = m1.min()
xmax = m1.max()
ymin = m2.min()
ymax = m2.max()
print(xmin, xmax, ymin, ymax)

In [ ]:
X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([m1, m2])
kernel = stats.gaussian_kde(values)
Z = np.reshape(kernel.evaluate(positions).T, X.shape)
print(Z.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r, extent=[xmin, xmax, ymin, ymax], origin='upper')
ax.plot(m1, m2, 'k.', markersize=2)
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
levels = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.11, 0.12, 0.13, 0.14, 0.15]
cs = ax.contour(X, Y, Z, levels=levels); # I dont't know what those levels mean... but it works fine!

In [ ]:
# We save the contour paths in a list
paths = []
for collec in cs.collections:
    try:
        paths.append(collec.get_paths()[0])
    except:
        pass

In [ ]:
# Looking for the number of points inside each contour
print(len(m1))
for level, path in zip(levels, paths):
    print('level {0:4.2f} contains {1:2.0f}% of the data'.format(level, 
                                path.contains_points(list(zip(m1, m2))).sum() / float(len(m1))*100))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(m1, m2, 'k.', markersize=2)
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
cs = ax.contour(X, Y, Z, levels=[0.08]); #  seems to correspond to 50% of the points inside

### Convolution

More information there: http://docs.scipy.org/doc/scipy/reference/tutorial/ndimage.html

In [ ]:
# Let's define an image representing a long slit of width 10 pixels
slit = np.zeros((100, 100))
slit[30:50, :] = 1

In [ ]:
plt.imshow(slit)
plt.colorbar();

In [ ]:
# This is the routine to apply a gaussian convolution
from scipy.ndimage.filters import gaussian_filter

In [ ]:
slit_seeing = gaussian_filter(slit, 3) # Convolve with a gaussian, 3 is the standard deviation in pixels
plt.imshow(slit_seeing)
plt.colorbar();

In [ ]:
f, ax =plt.subplots()
ax.plot(slit[:,50], label='slit') # original slit
ax.plot(slit_seeing[:,50], label='slit+seeing') # slit with seeing
ax.legend(loc='best');

In [ ]:
# Check that the slit transmission is conserved:
print(simps(slit[:,50]), simps(slit_seeing[:,50]))

### Quantiles

In [ ]:
from scipy.stats.mstats import mquantiles

In [ ]:
#help(mquantiles)

In [ ]:
data = np.random.randn(1000)

In [ ]:
mquantiles(data, [0.16, 0.84]) # should return something close to -1, 1 (the stv of the normal distribution)

In [ ]:
data = np.array([[   6.,    7.,    1.],
                         [  47.,   15.,    2.],
                         [  49.,   36.,    3.],
                         [  15.,   39.,    4.],
                         [  42.,   40., -999.],
                         [  41.,   41., -999.],
                         [   7., -999., -999.],
                         [  39., -999., -999.],
                         [  43., -999., -999.],
                         [  40., -999., -999.],
                         [  36., -999., -999.]])

In [ ]:
mq = mquantiles(data, axis=0, limit=(0, 50))
print(mq)
print(type(mq))
mq?
print(mq.mask)

### Input/Output

Scipy has many modules, classes, and functions available to read data from and write data to a variety of file formats.

Including MATLAB and IDL files. See http://docs.scipy.org/doc/scipy/reference/io.html